In [1]:
%%file spline.f90

SUBROUTINE spline(x,y,n,yp1,ypn,y2)
INTEGER n,NMAX
REAL yp1,ypn,x(n),y(n),y2(n)
PARAMETER (NMAX=500)
INTEGER i,k
REAL p,qn,sig,un,u(NMAX)
if (yp1.gt..99e30) then
  y2(1)=0.
  u(1)=0.
else
  y2(1)=-0.5
  u(1)=(3./(x(2)-x(1)))*((y(2)-y(1))/(x(2)-x(1))-yp1)
endif
do 11 i=2,n-1
  sig=(x(i)-x(i-1))/(x(i+1)-x(i-1))
  p=sig*y2(i-1)+2.
  y2(i)=(sig-1.)/p
  u(i)=(6.*((y(i+1)-y(i))/(x(i+1)-x(i))-(y(i)-y(i-1)) /(x(i)-x(i-1)))/(x(i+1)-x(i-1))-sig*u(i-1))/p
enddo 11
if (ypn.gt..99e30) then
  qn=0.
  un=0.
else
  qn=0.5
  un=(3./(x(n)-x(n-1)))*(ypn-(y(n)-y(n-1))/(x(n)-x(n-1)))
endif
y2(n)=(un-qn*u(n-1))/(qn*y2(n-1)+1.)
do 12 k=n-1,1,-1
  y2(k)=y2(k)*y2(k+1)+u(k)
enddo 12
return
END

Writing spline.f90


In [2]:
%%file splint.f90

SUBROUTINE splint(xa,ya,y2a,n,x,y)
INTEGER n
REAL x,y,xa(n),y2a(n),ya(n)

INTEGER k,khi,klo
REAL a,b,h
klo=1
khi=n
1 if (khi-klo.gt.1) then
k=(khi+klo)/2
if(xa(k).gt.x)then
khi=k
else
klo=k
endif
goto 1
endif
h=xa(khi)-xa(klo)
if (h.eq.0.) pause ’bad xa input in splint’
a=(xa(khi)-x)/h
b=(x-xa(klo))/h
y=a*ya(klo)+b*ya(khi)+((a**3-a)*y2a(klo)+(b**3-b)*y2a(khi))*(h**2)/6.
return
END

Writing splint.f90


In [3]:

from scipy import *
from matplotlib.pyplot import *
import weave
# define support code
with open("spline.c","r") as f:
  scode=f.read()
h=logspace(-3,0,16)
# h=input("Enter h as a numpy vector: ")
N=(2.0*pi)/h
err=zeros(h.shape)
for i in range(len(h)):
  x=linspace(0,2*pi,N[i])
  y=sin(x)
  n=int(N[i])
  xx=linspace(0,2*pi,10*n+1)
  y2=zeros(x.size)
  u=zeros(x.size)
  yy=zeros(xx.size)
  code="""
  #include<math.h>
  int i;
  double xp;
  spline(x,y,n,cos(x[0]),cos(x[n-1]),y2,u);
  for(i=0;i<=10*n;i++){
  xp=xx[i];
  splint(x,y,y2,n,xp,yy+i);
  }
"""
  weave.inline(code,["x","y","n","y2","u","xx","yy"],\
  support_code=scode,extra_compile_args=["-g"],\
  compiler="gcc")
  if i==0:
    figure(0)
    plot(x,y,"ro")
    plot(xx,yy,"g")
    title("Interpolated values and data points for n=%d" % N[i])
  z=abs(yy-sin(xx))
  err[i]=z.max()
figure(1)
loglog(h,err)
title("Error vs. spacing")
show()

d:\Anaconda\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


AttributeError: module 'distutils.sysconfig' has no attribute '_init_posix'

In [1]:
!pip install scipy-weave


     ---------------------------------------- 0.0/573.9 kB ? eta -:--:--
     -------------------------------------- 573.9/573.9 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\bprav\AppData\Local\Temp\pip-install-4ne8_h0b\scipy-weave_7d4c297c48b14d2bbfc45ccf0a087f5a\setup.py", line 32, in <module>
          raise RuntimeError("Python version 2.6 or 2.7 required.")
      RuntimeError: Python version 2.6 or 2.7 required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


  Using cached scipy-weave-0.18.0.tar.gz (573 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
